# Creating Issue Dimenion Table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_catalog.gold.dim_issue
(
  issue_id STRING NOT NULL PRIMARY KEY,
  complaint_number INT,
  consumer_consent_provided STRING,
  consumer_complaint_narrative STRING,
  date_sent_to_company DATE
)
USING DELTA
LOCATION 'abfss://gold@rcmadls10dev.dfs.core.windows.net/dim_issue';

In [0]:
%sql
-- select category_id, count(category_id) from databricks_catalog.gold.dim_category 
-- group by category_id
-- having count(category_id) > 1
-- ;

-- drop table databricks_catalog.gold.dim_category;

-- truncate table databricks_catalog.gold.dim_category;

In [0]:
%sql
WITH deduped_source AS (
  SELECT
    issue_id,
    FIRST(complaint_id) as complaint_id,
    FIRST(consumer_consent_provided) as consumer_consent_provided,
    FIRST(consumer_complaint_narrative) as consumer_complaint_narrative,
    FIRST(date_sent_to_company) as date_sent_to_company
  FROM databricks_catalog.silver.complaints
  WHERE issue_id IS NOT NULL
  GROUP BY issue_id
)

MERGE INTO databricks_catalog.gold.dim_issue TARGET
USING deduped_source SOURCE
ON TARGET.issue_id = SOURCE.issue_id
WHEN MATCHED THEN
  UPDATE SET 
    TARGET.complaint_number = SOURCE.complaint_id,
    TARGET.consumer_consent_provided = SOURCE.consumer_consent_provided,
    TARGET.consumer_complaint_narrative = SOURCE.consumer_complaint_narrative,
    TARGET.date_sent_to_company = SOURCE.date_sent_to_company
WHEN NOT MATCHED THEN
  INSERT (issue_id, complaint_number, consumer_consent_provided, consumer_complaint_narrative, date_sent_to_company)
  VALUES (SOURCE.issue_id, SOURCE.complaint_id, SOURCE.consumer_consent_provided, SOURCE.consumer_complaint_narrative, SOURCE.date_sent_to_company)

In [0]:
%sql
select * from databricks_catalog.gold.dim_issue;